# PaddleNLP词向量应用展示

In [5]:
#更新PaddleNLP
!pip install --upgrade paddlenlp -i https://pypi.org/simple

Requirement already up-to-date: paddlenlp in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (2.0.2)


## 加载TokenEmbedding

`TokenEmbedding()`参数
- `embedding_name`
将模型名称以参数形式传入TokenEmbedding，加载对应的模型。默认为`w2v.baidu_encyclopedia.target.word-word.dim300`的词向量。
- `unknown_token`
未知token的表示，默认为[UNK]。
- `unknown_token_vector`
未知token的向量表示，默认生成和embedding维数一致，数值均值为0的正态分布向量。
- `extended_vocab_path`
扩展词汇列表文件路径，词表格式为一行一个词。如引入扩展词汇列表，trainable=True。
- `trainable`
Embedding层是否可被训练。True表示Embedding可以更新参数，False为不可更新。默认为True。

采用动态图的方式去建模

[TokenEmbedding源码](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/paddlenlp/embeddings/token_embedding.py)

[训练好的词向量](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/paddlenlp/embeddings/constant.py)

In [6]:
from paddlenlp.embeddings import TokenEmbedding

# 初始化TokenEmbedding， 预训练embedding未下载时会自动下载并加载数据
token_embedding = TokenEmbedding(embedding_name="w2v.mixed-large.target.word-word.dim300")

# 查看token_embedding详情
print(token_embedding)

[2021-06-13 19:38:41,719] [    INFO] - Loading token embedding...
[2021-06-13 19:39:04,850] [    INFO] - Finish loading embedding vector.
[2021-06-13 19:39:04,853] [    INFO] - Token Embedding info:             
Unknown index: 1292481             
Unknown token: [UNK]             
Padding index: 1292482             
Padding token: [PAD]             
Shape :[1292483, 300]


Object   type: TokenEmbedding(1292483, 300, padding_idx=1292482, sparse=False)             
Unknown index: 1292481             
Unknown token: [UNK]             
Padding index: 1292482             
Padding token: [PAD]             
Parameter containing:
Tensor(shape=[1292483, 300], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[ 0.10238700,  0.11114600,  0.26331899, ..., -0.07657100,  0.14428701, -0.05603500],
        [ 0.08134800,  0.07354500,  0.20183100, ..., -0.00808900,  0.19217600, -0.04847400],
        [ 0.09659900,  0.08940500,  0.26543599, ..., -0.14453600,  0.13566500, -0.01968100],
        ...,
        [ 0.01123800, -0.01640600,  0.00053800, ...,  0.00284500,  0.00604800, -0.02247500],
        [-0.00205046,  0.01501678, -0.01676136, ...,  0.00834196,  0.01028977,  0.02585905],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,  0.        ,  0.        ]])


### 认识一下Embedding
**`TokenEmbedding.search()`**
获得指定词汇的词向量。

In [7]:
test_token_embedding = token_embedding.search("中国")
print(test_token_embedding)

[[ 1.46914e-01 -7.16610e-02  1.54542e-01  2.25867e-01  3.51830e-02
  -1.37828e-01  2.03395e-01  1.26151e-01  2.63094e-01  2.88970e-02
  -2.70334e-01  1.18251e-01  7.51060e-02  1.19036e-01  7.86690e-02
   4.07584e-01 -1.86970e-02  1.01131e-01 -4.82650e-02 -9.59120e-02
  -4.31150e-02  2.83825e-01  3.42903e-01 -6.79730e-02 -1.65933e-01
  -5.92840e-02 -1.92654e-01 -2.74950e-02  1.49001e-01  3.18610e-01
  -8.77040e-02 -4.03310e-02  9.01030e-02  1.60909e-01  1.39330e-02
   1.48743e-01  1.41160e-02 -1.78014e-01 -1.41943e-01  7.19590e-02
  -1.52930e-01 -9.76770e-02 -2.44922e-01  9.81100e-03 -8.31800e-03
  -1.56487e-01 -2.69268e-01  1.14820e-02  9.80550e-02 -3.25510e-02
   1.56373e-01  9.91290e-02 -1.02377e-01  6.66060e-02 -4.79992e-01
   4.84110e-02 -2.02188e-01 -3.16230e-01  1.16970e-02  1.24274e-01
   8.35290e-02  1.71044e-01  1.33700e-02  2.49910e-01  1.29088e-01
  -1.95874e-01 -4.46170e-02 -9.90400e-02 -1.23424e-01 -1.58375e-01
  -1.54670e-02  4.52074e-01 -2.73170e-01  7.38460e-02  1.73223

**`TokenEmbedding.cosine_sim()`**
计算词向量间余弦相似度，语义相近的词语余弦相似度更高，说明预训练好的词向量空间有很好的语义表示能力。

In [8]:
score1 = token_embedding.cosine_sim("女孩", "女人")
score2 = token_embedding.cosine_sim("女孩", "书籍")
score3 = token_embedding.cosine_sim("女孩", "男孩")
score4 = token_embedding.cosine_sim("女人", "男人")
print('score1:', score1)
print('score2:', score2)
print('score3:', score3)
print('score4:', score4)

score1: 0.6825842
score2: 0.2307212
score3: 0.8597522
score4: 0.8917782


### 词向量映射到低维空间

使用深度学习可视化工具[VisualDL](https://github.com/PaddlePaddle/VisualDL)的[High Dimensional](https://github.com/PaddlePaddle/VisualDL/blob/develop/docs/components/README_CN.md#High-Dimensional--%E6%95%B0%E6%8D%AE%E9%99%8D%E7%BB%B4%E7%BB%84%E4%BB%B6)组件可以对embedding结果进行可视化展示，便于对其直观分析，步骤如下：

1. 升级 VisualDL 最新版本。

`pip install --upgrade visualdl`

2. 创建LogWriter并将记录词向量。

3. 点击左侧面板中的可视化tab，选择‘token_hidi’作为文件并启动VisualDL可视化

In [ ]:
!pip install --upgrade visualdl

In [ ]:
# 获取词表中前1000个单词
labels = token_embedding.vocab.to_tokens(list(range(0, 1000)))
# 取出这1000个单词对应的Embedding
test_token_embedding = token_embedding.search(labels)

# 引入VisualDL的LogWriter记录日志
from visualdl import LogWriter

with LogWriter(logdir='./token_hidi') as writer:
    writer.add_embeddings(tag='test', mat=[i for i in test_token_embedding], metadata=labels)

#### 启动VisualDL查看词向量降维效果
启动步骤：
- 1、切换到「可视化」指定可视化日志
- 2、日志文件选择 'token_hidi'
- 3、点击「启动VisualDL」后点击「打开VisualDL」，选择「高维数据映射」，即可查看词表中前1000词UMAP方法下映射到三维空间的可视化结果:

![](https://user-images.githubusercontent.com/48054808/120594172-1fe02b00-c473-11eb-9df1-c0206b07e948.gif)

可以看出，语义相近的词在词向量空间中聚集(如数字、章节等)，说明预训练好的词向量有很好的文本表示能力。

使用VisualDL除可视化embedding结果外，还可以对标量、图片、音频等进行可视化，有效提升训练调参效率。关于VisualDL更多功能和详细介绍，可参考[VisualDL使用文档](https://github.com/PaddlePaddle/VisualDL/tree/develop/docs)。

![](https://ai-studio-static-online.cdn.bcebos.com/b6d27d099f5147aeae4c87a7f51409b1b831706ca3d1471c9edfef55556a6b3d)


## 基于TokenEmbedding衡量句子语义相似度

在许多实际应用场景（如文档检索系统）中， 需要衡量两个句子的语义相似程度。此时我们可以使用词袋模型（Bag of Words，简称BoW）计算句子的语义向量。

**首先**，将两个句子分别进行切词，并在TokenEmbedding中查找相应的单词词向量（word embdding）。

**然后**，根据词袋模型，将句子的word embedding叠加作为句子向量（sentence embedding）。

**最后**，计算两个句子向量的余弦相似度。

### 基于TokenEmbedding的词袋模型


使用`BoWEncoder`搭建一个BoW模型用于计算句子语义。

* `paddlenlp.TokenEmbedding`组建word-embedding层
* `paddlenlp.seq2vec.BoWEncoder`组建句子建模层


In [9]:
import paddle
import paddle.nn as nn
import paddlenlp


class BoWModel(nn.Layer):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder
        emb_dim = self.embedder.embedding_dim
        # 沿着1轴将向量相加
        self.encoder = paddlenlp.seq2vec.BoWEncoder(emb_dim)
        self.cos_sim_func = nn.CosineSimilarity(axis=-1)

    def get_cos_sim(self, text_a, text_b):
        text_a_embedding = self.forward(text_a)
        text_b_embedding = self.forward(text_b)
        cos_sim = self.cos_sim_func(text_a_embedding, text_b_embedding)
        return cos_sim

    def forward(self, text):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, embedding_dim)
        summed = self.encoder(embedded_text)

        return summed

model = BoWModel(embedder=token_embedding)

### 构造Tokenizer
使用TokenEmbedding词表构造Tokenizer。

In [10]:
from data import Tokenizer
tokenizer = Tokenizer()
tokenizer.set_vocab(vocab=token_embedding.vocab)

### 相似句对数据读取

以提供的样例数据text_pair.txt为例，该数据文件每行包含两个句子。


In [11]:
text_pairs = {}
with open("text_pair.txt", "r", encoding="utf8") as f:
    for line in f:
        text_a, text_b = line.strip().split("\t")
        if text_a not in text_pairs:
            text_pairs[text_a] = []
        text_pairs[text_a].append(text_b)

### 查看相似语句相关度

In [12]:
for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        print("text_a: {}".format(text_a))
        print("text_b: {}".format(text_b))
        print("cosine_sim: {}".format(model.get_cos_sim(text_a_ids, text_b_ids).numpy()[0]))
        print()

text_a: 我喜欢吃饭
text_b: 我喜欢吃东西
cosine_sim: 0.915885865688324

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 多项式矩阵的左共轭积及其应用
cosine_sim: 0.9169095754623413

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 退化阻尼对高维可压缩欧拉方程组经典解的影响
cosine_sim: 0.8052490949630737

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Burgers方程基于特征正交分解方法的数值解法研究
cosine_sim: 0.8311429023742676

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 有界对称域上解析函数空间的若干性质
cosine_sim: 0.8142803311347961

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于卷积神经网络的图像复杂度研究与应用
cosine_sim: 0.7054457664489746

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: Cartesian发射机中线性功率放大器的研究
cosine_sim: 0.6882213950157166

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: CFRP加固WF型梁侧扭屈曲的几何非线性有限元分析
cosine_sim: 0.7472860217094421

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于线性CCD自适应成像的光刻机平台调平方法研究
cosine_sim: 0.6940180659294128

text_a: 多项式矩阵左共轭积对偶Sylvester共轭和数学算子完备参数解
text_b: 基于变分贝叶斯理论的图像复原方法研究
cosine_sim: 0.6928799152374268

text_a: 多项式矩阵左共

### 使用VisualDL查看句子向量

In [ ]:
# 引入VisualDL的LogWriter记录日志
import numpy as np
from visualdl import LogWriter    
# 获取句子以及其对应的向量
label_list = []
embedding_list = []

for text_a, text_b_list in text_pairs.items():
    text_a_ids = paddle.to_tensor([tokenizer.text_to_ids(text_a)])
    embedding_list.append(model(text_a_ids).flatten().numpy())
    label_list.append(text_a)

    for text_b in text_b_list:
        text_b_ids = paddle.to_tensor([tokenizer.text_to_ids(text_b)])
        embedding_list.append(model(text_b_ids).flatten().numpy())
        label_list.append(text_b)


with LogWriter(logdir='./sentence_hidi') as writer:
    writer.add_embeddings(tag='test', mat=embedding_list, metadata=label_list)

### 启动VisualDL观察句子向量降维效果

步骤如上述观察词向量降维效果一模一样。
![](https://ai-studio-static-online.cdn.bcebos.com/0e876f3cf1724e90a317ad3f4be233a9eb0313b0e92f475b95675c2ad52d3eb0)


可以看出，语义相近的句子在句子向量空间中聚集(如有关课堂的句子、有关化学描述句子等)。

![](https://ai-studio-static-online.cdn.bcebos.com/c1f4cbefec724f0b9dabaa5e82c3ebe28c4ff9bddc92406f901ca8750ba9d6f0)


# 预训练词向量辅助分类任务

想学习词向量更多应用，来试试预训练词向量对分类模型的改善效果吧，[这里](https://aistudio.baidu.com/aistudio/projectdetail/1283423) 试试把`paddle.nn.Embedding`换成刚刚学到的预训练词向量吧。